In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.utils import shuffle

Format dependent variable of toy dataset by:
1. Getting party position for each bill (Whichever recieves more votes)
2. Compare that to member vote
3. Split into Train/Test set

In [186]:
df_votes = pd.read_csv('../propublica/member_votes.csv', encoding='latin1')
#df_bills = pd.read_csv('../propublica/member_votes_bills.csv', encoding='latin1')


In [187]:
df_members_house = pd.read_csv('../propublica/members_house.csv', encoding='latin1')
df_members_senate = pd.read_csv('../propublica/members_senate.csv', encoding = 'latin1')
df_members = pd.concat([df_members_senate[['id','session','party']], 
                        df_members_house[['id','session','party']]])
df_members['id_ses'] = df_members.id + df_members.session.apply(lambda x: str(x))

#filter out members who changed parties within a single congressional session
switch_party = pd.DataFrame(df_members.groupby(['id', 'session']).party.size())
switch_party = switch_party[switch_party[0] > 1]
switch_party = switch_party.reset_index()
switch_party['id_ses'] = switch_party.id + switch_party.session.apply(lambda x: str(x))
df_members = df_members[df_members.id_ses.isin(switch_party.id_ses.tolist()) == False]

In [188]:
df_votes = pd.merge(df_votes, df_members, how = 'left', left_on=['member_id', 'congress'], right_on = ['id','session'])

In [189]:
df_votes.rename(columns={'bill_id':'bill_slug'}, inplace = True)
df_votes['bill_id'] = df_votes.bill_slug + "-" + df_votes.congress.apply(lambda x: str(x))
sufficient_sample = df_votes.bill_id.value_counts()
sufficient_sample = sufficient_sample[sufficient_sample > 60].index.tolist()
df_votes = df_votes[df_votes.bill_id.isin(sufficient_sample)]

In [190]:
vote_majority = pd.DataFrame(df_votes.groupby(['bill_id','party','position']).size()).unstack().apply(lambda x: x.argmax()[1], axis = 1)
vote_majority = pd.DataFrame(vote_majority).rename(columns = {0:'party_position_'}).unstack()
vote_majority = vote_majority.reset_index()
df_votes = pd.merge(df_votes, vote_majority, how = 'left', on = 'bill_id')
for party in ('R','D'):
    c1 = (df_votes[('party_position_', party)] == 'Yes')
    c2 = (df_votes[('party_position_', party)] == 'No')
    df_votes = df_votes[c1 | c2]
df_votes = df_votes[df_votes.party.isin(['D','R'])]

C:\Users\Steve\Anaconda3\lib\site-packages\pandas\tools\merge.py:536: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Steve\Anaconda3\lib\site-packages\pandas\tools\merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [191]:
def voted_against_party(row):
    party = row.party
    if row[('party_position_', party)] != row.position:
        return 1
    else:
        return 0

df_votes['voted_against_party'] = df_votes.apply(voted_against_party, axis = 1)
columns = ['id', 'bill_id','date','party','voted_against_party','position','sponsor_id']
df_votes = df_votes[columns]

df_votes = shuffle(df_votes)
source = ['Train'] * int(len(df_votes) * .75)
source = source + (['Test'] * (len(df_votes) - len(source)))
df_votes['source'] = source

In [192]:
df_votes.tail()

,id,bill_id,date,party,voted_against_party,position,sponsor_id,source
3072,C000293,hres601-102,1992-10-05,R,1,Not Voting,NaN,Test
15029,O000001,hres609-102,1992-10-05,D,0,Yes,NaN,Test
5374,D000592,hr776-102,1992-10-05,D,0,Yes,NaN,Test
7106,F000457,pn371-115,2017-10-31,D,0,No,NaN,Test
10255,J000255,hres609-115,2017-11-08,R,0,Yes,B001297,Test


In [193]:
df_votes[df_votes.position.isin(['Yes','No'])].to_csv('../propublica/toyset.csv', index = False)

In [177]:
df_votes[df_votes.position.isin(['Yes','No'])].voted_against_party.mean()

0.1774757281553398